In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Import the libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

load the data from csv file.

In [ ]:
data = pd.read_csv('/kaggle/input/used-car-dataset-ford-and-mercedes/hyundi.csv')

In [ ]:
data.head(10)

Categories of Variables

In [ ]:
print(data.model.unique())
print(data.year.unique())
print(data.transmission.unique())
print(data.fuelType.unique())

Visualize and understand the data.

In [ ]:
plt.figure(figsize=(14,6))
sns.countplot(x='model',data=data)

In [ ]:
plt.figure(figsize=(14,6))
sns.countplot(x='year',data=data)

In [ ]:
plt.figure(figsize=(8,6))
sns.countplot(x='transmission',data=data)

In [ ]:
plt.figure(figsize=(8,6))
sns.countplot(x='fuelType',data=data)

In [ ]:
data.describe()

In [ ]:
sns.heatmap(data.corr(), annot=True, linewidths=1)

In [ ]:
plt.scatter(data.mileage, data.price)

In [ ]:
plt.scatter(data.year, data.price)

In [ ]:
plt.scatter(data['tax(£)'], data.price)

In [ ]:
plt.scatter(data.mpg, data.price)

In [ ]:
plt.scatter(data.engineSize, data.price)

From the visualization and the correlation of the data. tax and mpg are not contributing to predict the price. so, we can drop them.

In [ ]:
datac=data.copy()

datac.head()

One Hot Encode the categorical variables of the data for better performance.

In [ ]:
datac_d = pd.get_dummies(datac)
datac_d.head()

drop tax, mpg, and price from copied data

In [ ]:
datac_d=datac_d.drop(["tax(£)","mpg","price"], axis=1)

datac_d.head()

price column from the data 

In [ ]:
price = datac.price
price.head()

scale the numerical features by using min-max-scaler

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

data_scaled = scaler.fit_transform(datac_d)
data_scaled = pd.DataFrame(data_scaled, columns=datac_d.columns)

data_scaled.head()

split the data

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train, y_test = train_test_split(data_scaled, price, test_size=0.2, random_state=1)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

Try Linear Regression model

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(x_train,y_train)
preds = model.predict(x_test)
model.score(x_test, y_test)

The acuuracy is 87% which is pretty less. so lets try Random Forest Regressor because we have categorical columns.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

reg = RandomForestRegressor()
reg.fit(x_train,y_train)
reg.score(x_test, y_test)

The accuracy is 94%. 

features miliage and year had a polynomial relation with price.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures()
x_train_poly = poly.fit_transform(x_train)
x_test_poly = poly.fit_transform(x_test)

print(x_train.shape)
print(x_test.shape)

In [ ]:
regp = RandomForestRegressor()

regp.fit(x_train_poly, y_train)
preds_p = regp.predict(x_test_poly)
print(regp.score(x_test_poly, y_test))

The accuracy is pretty much same. 

now create a data frame to compare the predicted results

In [ ]:
predictions = pd.DataFrame({"price prediction": preds_p.flatten()})
y_test = y_test.reset_index()
result = y_test.join(predictions)
result = result.drop(['index'], axis=1)
result.head(10)